In [ ]:
from datasets import load_dataset,Audio
ds = load_dataset("PolyAI/minds14", name="en-US", split="train")
ds = ds.cast_column("audio", Audio(sampling_rate=16000))
ds[0]["audio"]
# {'array': array([ 2.3443763e-05,  2.1729663e-04,  2.2145823e-04, ...,
#      3.8356509e-05, -7.3497440e-06, -2.1754686e-05], dtype=float32),
#  'path': '/root/.cache/huggingface/datasets/downloads/extracted/f14948e0e84be638dd7943ac36518a4cf3324e8b7aa331c5ab11541518e9368c/en-US~JOINT_ACCOUNT/602ba55abb1e6d0fbce92065.wav',
#  'sampling_rate': 16000}


 ```py
    >>> from datasets import Features
    >>> features = Features({'stars': Value(dtype='int32')})
    >>> features
    {'stars': Value(dtype='int32', id=None)}
```

 ```py
    >>> from datasets import Features
    >>> features = Features({'x': Array2D(shape=(1, 3), dtype='int32')})
```

 ```py
    >>> from datasets import Features
    >>> features = Features({'x': Array3D(shape=(1, 2, 3), dtype='int32')})
```

```py
    >>> from datasets import Features
    >>> features = Features({'x': Array4D(shape=(1, 2, 2, 3), dtype='int32')})
```


```py
>>> from datasets import Features
>>> features = Features({'x': Array5D(shape=(1, 2, 2, 3, 3), dtype='int32')})
```

```py
>>> from datasets import Features
>>> features = Features({'label': ClassLabel(num_classes=3, names=['bad', 'ok', 'good'])})
>>> features
{'label': ClassLabel(num_classes=3, names=['bad', 'ok', 'good'], id=None)}
```

 ```py
>>> from datasets import load_dataset
>>> ds = load_dataset("rotten_tomatoes", split="train")
>>> ds.features["label"].str2int('neg')
0
```

```py
>>> from datasets import load_dataset
>>> ds = load_dataset("rotten_tomatoes", split="train")
>>> ds.features["label"].int2str(0)
'neg'
```

```py
>>> from datasets import Features, Sequence, Value, ClassLabel
>>> features = Features({'post': Sequence(feature={'text': Value(dtype='string'), 'upvotes': Valu(dtype='int32'), 'label': ClassLabel(num_classes=2, names=['hot', 'cold'])})})
>>> features
{'post': Sequence(feature={'text': Value(dtype='string', id=None), 'upvotes': Valu(dtype='int32', id=None), 'label': ClassLabel(num_classes=2, names=['hot', 'cold'], id=None)},length=-1, id=None)}
```

```python
  >>> Features.from_dict({'_type': {'dtype': 'string', 'id': None, '_type': 'Value'}})
            {'_type': Value(dtype='string', id=None)}
```

 ```py
>>> from datasets import load_dataset
>>> ds = load_dataset("rotten_tomatoes", split="train")
>>> copy_of_features = ds.features.copy()
>>> copy_of_features
        {'label': ClassLabel(num_classes=2, names=['neg', 'pos'], id=None),
         'text': Value(dtype='string', id=None)}
```

```python
>>> from datasets import Features, Sequence, Value
>>> # let's say we have to features with a different order of nested fields (for a and b for example)
>>> f1 = Features({"root": Sequence({"a": Value("string"), "b": Value("string")})})
>>> f2 = Features({"root": {"b": Sequence(Value("string")), "a": Sequence(Value("string"))}})
>>> assert f1.type != f2.type
>>> # re-ordering keeps the base structure (here Sequence is defined at the root level), but make the fields order match
>>> f1.reorder_fields_as(f2)
            {'root': Sequence(feature={'b': Value(dtype='string', id=None), 'a': Value(dtype='string', id=None)}, length=-1, id=None)}
>>> assert f1.reorder_fields_as(f2).type == f2.type
```

```py
>>> from datasets import load_dataset
>>> ds = load_dataset("squad", split="train")
>>> ds.features.flatten()
{'answers.answer_start': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'answers.text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'context': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None)}
```

```python
    >>> # At construction time:
    >>> datasets.features.Translation(languages=['en', 'fr', 'de'])
    >>> # During data generation:
    >>> yield {
    ...         'en': 'the cat',
    ...         'fr': 'le chat',
    ...         'de': 'die katze'
    ... }
```

```python

    >>> # At construction time:
    >>> datasets.features.TranslationVariableLanguages(languages=['en', 'fr', 'de'])
    >>> # During data generation:
    >>> yield {
    ...         'en': 'the cat',
    ...         'fr': ['le chat', 'la chatte,']
    ...         'de': 'die katze'
    ... }
    >>> # Tensor returned :
    >>> {
    ...         'language': ['en', 'de', 'fr', 'fr'],
    ...         'translation': ['the cat', 'die katze', 'la chatte', 'le chat'],
    ... }
```

Let me provide an improved example that addresses the drawbacks you mentioned and demonstrates a more flexible and modular approach to loading and preprocessing datasets using the Datasets library from Hugging Face:

```python
from datasets import load_dataset, Dataset
from typing import List, Dict, Any, Callable
import torchvision.transforms as transforms
from PIL import Image

def load_dataset_with_config(dataset_name: str, subset: str, preprocessing_config: Dict[str, Any]) -> Dataset:
    """
    Load a dataset from Hugging Face with a specified preprocessing configuration.
    
    Args:
        dataset_name (str): Name of the dataset to load.
        subset (str): Subset of the dataset to use (e.g., "train", "validation", "test").
        preprocessing_config (Dict[str, Any]): Configuration for preprocessing the dataset.
        
    Returns:
        Dataset: Loaded dataset with the specified preprocessing configuration.
    """
    dataset = load_dataset(dataset_name, split=subset)
    preprocessed_dataset = dataset.map(
        lambda example: preprocess_example(example, preprocessing_config),
        batched=True,
        remove_columns=dataset.column_names,
    )
    return preprocessed_dataset

def preprocess_example(example: Dict[str, Any], preprocessing_config: Dict[str, Any]) -> Dict[str, Any]:
    """
    Preprocess a single example from the dataset based on the provided configuration.
    
    Args:
        example (Dict[str, Any]): Input example from the dataset.
        preprocessing_config (Dict[str, Any]): Configuration for preprocessing the example.
        
    Returns:
        Dict[str, Any]: Preprocessed example.
    """
    preprocessed_example = {}
    
    for field, config in preprocessing_config.items():
        if field in example:
            preprocessor = get_preprocessor(config["type"])
            preprocessed_example[field] = preprocessor(example[field], **config["args"])
    
    return preprocessed_example

def get_preprocessor(preprocessor_type: str) -> Callable:
    """
    Get the preprocessor function based on the specified type.
    
    Args:
        preprocessor_type (str): Type of the preprocessor.
        
    Returns:
        Callable: Preprocessor function.
    """
    if preprocessor_type == "text_cleaning":
        return preprocess_text
    elif preprocessor_type == "image_transformation":
        return preprocess_image
    else:
        raise ValueError(f"Unknown preprocessor type: {preprocessor_type}")

def preprocess_text(text: str, **kwargs) -> str:
    """
    Preprocess text data.
    
    Args:
        text (str): Input text.
        **kwargs: Additional arguments for text preprocessing.
        
    Returns:
        str: Preprocessed text.
    """
    # Perform text preprocessing steps
    text = text.lower()
    text = "".join(char for char in text if char.isalnum() or char.isspace())
    return text

def preprocess_image(image_path: str, **kwargs) -> Any:
    """
    Preprocess image data.
    
    Args:
        image_path (str): Path to the input image.
        **kwargs: Additional arguments for image preprocessing.
        
    Returns:
        Any: Preprocessed image.
    """
    # Perform image preprocessing steps
    image = Image.open(image_path)
    transform = transforms.Compose([
        transforms.Resize((kwargs["image_size"], kwargs["image_size"])),
        transforms.ToTensor(),
        transforms.Normalize(mean=kwargs["mean"], std=kwargs["std"])
    ])
    image = transform(image)
    return image

# Example usage
dataset_name = "my_dataset"
subset = "train"
preprocessing_config = {
    "text": {
        "type": "text_cleaning",
        "args": {}
    },
    "image": {
        "type": "image_transformation",
        "args": {
            "image_size": 224,
            "mean": [0.485, 0.456, 0.406],
            "std": [0.229, 0.224, 0.225]
        }
    }
}

preprocessed_dataset = load_dataset_with_config(dataset_name, subset, preprocessing_config)
print(preprocessed_dataset)
```

In this improved example:

1. We define a function `load_dataset_with_config` that takes the dataset name, subset, and a preprocessing configuration dictionary as arguments. This allows for more flexibility in specifying different preprocessing steps for different fields of the dataset.
2. Inside the function, we load the dataset using `load_dataset` and then apply the `preprocess_example` function to each example using the `map` function from the Datasets library. The `remove_columns` parameter is used to remove the original columns from the dataset after preprocessing.
3. The `preprocess_example` function takes an example and the preprocessing configuration as arguments. It iterates over the fields specified in the configuration and applies the corresponding preprocessor function to each field.
4. The `get_preprocessor` function is used to retrieve the appropriate preprocessor function based on the specified preprocessor type in the configuration.
5. The `preprocess_text` and `preprocess_image` functions are separate preprocessors for text and image data, respectively. They take the input data and any additional arguments specified in the configuration.
6. The preprocessing configuration is defined as a dictionary that specifies the preprocessing steps for each field of the dataset. In this example, we have a configuration for the "text" field with a "text_cleaning" preprocessor and an "image" field with an "image_transformation" preprocessor.
7. Finally, we call the `load_dataset_with_config` function with the dataset name, subset, and preprocessing configuration to obtain the preprocessed dataset.

This approach provides more flexibility and modularity in defining and applying preprocessing steps to different fields of the dataset. It avoids hardcoding and allows for easier customization and extension of the preprocessing pipeline.

Apologies for the confusion. Let me provide a more comprehensive and advanced end-to-end pipeline example using the Datasets library from Hugging Face and incorporating various functionalities from torchvision:

```python
from datasets import load_dataset
from typing import List, Dict, Any, Callable, Union
import torchvision.transforms as transforms
from PIL import Image
import torch

def load_and_preprocess_dataset(
    dataset_name: str,
    split: Union[str, List[str]],
    preprocessing_config: Dict[str, Any],
    batch_size: int = 1,
    num_workers: int = 0,
    shuffle: bool = True,
) -> torch.utils.data.DataLoader:
    """
    Load a dataset from Hugging Face and perform preprocessing.
    
    Args:
        dataset_name (str): Name of the dataset to load.
        split (Union[str, List[str]]): Split(s) of the dataset to use.
        preprocessing_config (Dict[str, Any]): Configuration for preprocessing the dataset.
        batch_size (int): Number of examples to process in each batch.
        num_workers (int): Number of worker processes for data loading.
        shuffle (bool): Whether to shuffle the dataset.
        
    Returns:
        torch.utils.data.DataLoader: DataLoader for the preprocessed dataset.
    """
    dataset = load_dataset(dataset_name, split=split)
    
    def preprocess_function(examples: Dict[str, List]) -> Dict[str, Union[List, torch.Tensor]]:
        """
        Preprocessing function to be applied to each batch of examples.
        
        Args:
            examples (Dict[str, List]): Batch of examples from the dataset.
            
        Returns:
            Dict[str, Union[List, torch.Tensor]]: Preprocessed batch of examples.
        """
        preprocessed_examples = {}
        
        for column, config in preprocessing_config.items():
            if column in examples:
                preprocessor = get_preprocessor(config["type"])
                preprocessed_examples[column] = preprocessor(examples[column], **config["args"])
        
        return preprocessed_examples
    
    preprocessed_dataset = dataset.map(
        preprocess_function,
        batched=True,
        batch_size=batch_size,
        remove_columns=dataset.column_names,
        num_proc=num_workers,
    )
    
    preprocessed_dataset.set_format(type="torch", columns=list(preprocessing_config.keys()))
    
    dataloader = torch.utils.data.DataLoader(
        preprocessed_dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
    )
    
    return dataloader

def get_preprocessor(preprocessor_type: str) -> Callable:
    """
    Get the preprocessor function based on the specified type.
    
    Args:
        preprocessor_type (str): Type of the preprocessor.
        
    Returns:
        Callable: Preprocessor function.
    """
    if preprocessor_type == "text_preprocessing":
        return preprocess_text
    elif preprocessor_type == "image_preprocessing":
        return preprocess_image
    else:
        raise ValueError(f"Unknown preprocessor type: {preprocessor_type}")

def preprocess_text(texts: List[str], **kwargs) -> List[str]:
    """
    Preprocess text data.
    
    Args:
        texts (List[str]): List of input texts.
        **kwargs: Additional arguments for text preprocessing.
        
    Returns:
        List[str]: List of preprocessed texts.
    """
    # Perform text preprocessing steps
    preprocessed_texts = [
        "".join(char for char in text.lower() if char.isalnum() or char.isspace())
        for text in texts
    ]
    return preprocessed_texts

def preprocess_image(image_paths: List[str], **kwargs) -> torch.Tensor:
    """
    Preprocess image data.
    
    Args:
        image_paths (List[str]): List of paths to the input images.
        **kwargs: Additional arguments for image preprocessing.
        
    Returns:
        torch.Tensor: Preprocessed images.
    """
    # Perform image preprocessing steps
    images = [Image.open(path) for path in image_paths]
    
    transform = transforms.Compose([
        transforms.Resize((kwargs["image_size"], kwargs["image_size"])),
        transforms.RandomCrop(kwargs["crop_size"]),
        transforms.RandomHorizontalFlip(p=kwargs["flip_prob"]),
        transforms.RandomRotation(kwargs["rotation_range"]),
        transforms.ColorJitter(
            brightness=kwargs["brightness"],
            contrast=kwargs["contrast"],
            saturation=kwargs["saturation"],
            hue=kwargs["hue"]
        ),
        transforms.ToTensor(),
        transforms.Normalize(mean=kwargs["mean"], std=kwargs["std"]),
    ])
    
    preprocessed_images = torch.stack([transform(image) for image in images])
    return preprocessed_images

# Example usage
dataset_name = "my_dataset"
split = ["train", "validation"]
preprocessing_config = {
    "text": {
        "type": "text_preprocessing",
        "args": {}
    },
    "image": {
        "type": "image_preprocessing",
        "args": {
            "image_size": 256,
            "crop_size": 224,
            "flip_prob": 0.5,
            "rotation_range": 30,
            "brightness": 0.2,
            "contrast": 0.2,
            "saturation": 0.2,
            "hue": 0.1,
            "mean": [0.485, 0.456, 0.406],
            "std": [0.229, 0.224, 0.225],
        }
    }
}
batch_size = 32
num_workers = 4
shuffle = True

dataloader = load_and_preprocess_dataset(
    dataset_name, split, preprocessing_config, batch_size, num_workers, shuffle
)

for batch in dataloader:
    # Process the batch
    print(batch)
```

In this advanced end-to-end pipeline example:

1. The `load_and_preprocess_dataset` function now returns a `torch.utils.data.DataLoader` object, which provides an iterable over the preprocessed dataset.
2. The `split` parameter can be either a single split or a list of splits to load from the dataset.
3. The `num_workers` parameter is added to control the number of worker processes for data loading, enabling parallel processing.
4. The `shuffle` parameter is added to control whether to shuffle the dataset during iteration.
5. Inside the `preprocess_function`, the preprocessed examples are stored as a dictionary with columns as keys and preprocessed data as values.
6. The `set_format` function is used to convert the preprocessed dataset to a PyTorch format, specifying the columns to include.
7. The `torch.utils.data.DataLoader` is created with the preprocessed dataset, batch size, shuffle, and num_workers parameters.
8. The `preprocess_image` function now includes various image transformations from torchvision, such as random cropping, flipping, rotation, color jittering, and normalization.
9. The preprocessing configuration is updated to include additional parameters for image preprocessing, such as crop size, flip probability, rotation range, color jittering settings, and normalization parameters.

This advanced pipeline provides a comprehensive example of loading a dataset from Hugging Face, applying extensive preprocessing steps using torchvision functionalities, and creating a PyTorch DataLoader for efficient batch processing.

You can further customize the preprocessing steps, add more preprocessing options, and integrate additional functionalities as needed for your specific use case.

Class: __class__
  Documentation: str(object='') -> str
str(bytes_or_buffer[, encoding[, errors]]) -> str

Create a new string object from the given object. If encoding or
errors is specified, then the object must expose a data buffer
that will be decoded using the given encoding and error handler.
Otherwise, returns the result of object.__str__() (if defined)
or repr(object).
encoding defaults to sys.getdefaultencoding().
errors defaults to 'strict'.
  Constructor:
    Signature: (self, /, *args, **kwargs)
  Methods:


In [ ]:
import inspect
import importlib

def get_module_info(module_name, recursive=True, level=0):
    """
    Retrieves information about classes, functions, and submodules in a module.
    
    Args:
        module_name (str): The name of the module to inspect.
        recursive (bool): Whether to recursively explore submodules. Default is True.
        level (int): The indentation level for displaying information. Default is 0.
    
    Returns:
        None
    """
    module = importlib.import_module(module_name)
    indent = "  " * level
    
    print(f"{indent}Module: {module_name}")
    print(f"{indent}  Documentation: {module.__doc__}")
    
    for name, obj in inspect.getmembers(module):
        if inspect.isclass(obj):
            print(f"{indent}  Class: {name}")
            print(f"{indent}    Documentation: {obj.__doc__}")
            
            # Get constructor information
            print(f"{indent}    Constructor:")
            constructor_signature = inspect.signature(obj.__init__)
            print(f"{indent}      Signature: {constructor_signature}")
            
            # Get method information
            print(f"{indent}    Methods:")
            for method_name, method_obj in inspect.getmembers(obj, predicate=inspect.isfunction):
                print(f"{indent}      {method_name}:")
                print(f"{indent}        Documentation: {method_obj.__doc__}")
                method_signature = inspect.signature(method_obj)
                print(f"{indent}        Signature: {method_signature}")
            
            # Get class attribute information
            print(f"{indent}    Attributes:")
            for attr_name in obj.__dict__:
                if not attr_name.startswith("__"):
                    attr_value = getattr(obj, attr_name)
                    print(f"{indent}      {attr_name}: {attr_value}")
            
        elif inspect.isfunction(obj):
            print(f"{indent}  Function: {name}")
            print(f"{indent}    Documentation: {obj.__doc__}")
            function_signature = inspect.signature(obj)
            print(f"{indent}    Signature: {function_signature}")
        
        elif inspect.ismodule(obj) and recursive:
            submodule_name = f"{module_name}.{name}"
            get_module_info(submodule_name, recursive, level + 1)

# Example usage
import torch 
get_module_info("torch")


In [5]:
import inspect
import importlib
import os
import sys

def get_module_info(module_name, project_root, recursive=True, level=0):
    """
    Retrieves information about classes, functions, and submodules in a module.
    
    Args:
        module_name (str): The name of the module to inspect.
        project_root (str): The root directory of the project.
        recursive (bool): Whether to recursively explore submodules. Default is True.
        level (int): The indentation level for displaying information. Default is 0.
    
    Returns:
        None
    """
    try:
        module = importlib.import_module(module_name)
    except ImportError:
        print(f"Failed to import module: {module_name}")
        return
    
    indent = "  " * level
    
    print(f"{indent}Module: {module_name}")
    print(f"{indent}  Documentation: {module.__doc__}")
    
    for name, obj in inspect.getmembers(module):
        if inspect.isclass(obj):
            print(f"{indent}  Class: {name}")
            print(f"{indent}    Documentation: {obj.__doc__}")
            
            # Get constructor information
            try:
                constructor_signature = inspect.signature(obj.__init__)
                print(f"{indent}    Constructor:")
                print(f"{indent}      Signature: {constructor_signature}")
            except (ValueError, TypeError):
                print(f"{indent}    Constructor: Not found")
            
            # Get method information
            print(f"{indent}    Methods:")
            for method_name, method_obj in inspect.getmembers(obj, predicate=inspect.isfunction):
                print(f"{indent}      {method_name}:")
                print(f"{indent}        Documentation: {method_obj.__doc__}")
                try:
                    method_signature = inspect.signature(method_obj)
                    print(f"{indent}        Signature: {method_signature}")
                except (ValueError, TypeError):
                    print(f"{indent}        Signature: Not found")
            
            # Get class attribute information
            print(f"{indent}    Attributes:")
            for attr_name in obj.__dict__:
                if not attr_name.startswith("__"):
                    attr_value = getattr(obj, attr_name)
                    print(f"{indent}      {attr_name}: {attr_value}")
            
        elif inspect.isfunction(obj):
            print(f"{indent}  Function: {name}")
            print(f"{indent}    Documentation: {obj.__doc__}")
            try:
                function_signature = inspect.signature(obj)
                print(f"{indent}    Signature: {function_signature}")
            except (ValueError, TypeError):
                print(f"{indent}    Signature: Not found")
        
        elif inspect.ismodule(obj) and recursive:
            submodule_name = f"{module_name}.{name}"
            get_module_info(submodule_name, project_root, recursive, level + 1)

def explore_project(project_root):
    """
    Explores the project structure and retrieves information about modules.
    
    Args:
        project_root (str): The root directory of the project.
    
    Returns:
        None
    """
    sys.path.append(project_root)
    
    for root, dirs, files in os.walk(project_root):
        for file in files:
            if file.endswith(".py") and not file.startswith("__"):
                module_name = file[:-3]
                module_path = os.path.relpath(root, project_root).replace(os.sep, ".")
                if module_path != ".":
                    module_name = f"{module_path}.{module_name}"
                get_module_info(module_name, project_root)

# Example usage
project_root = "torch"
explore_project(project_root)


In [ ]:
import inspect

def print_module_info(module):
    """
    Prints information about classes, functions, and their input arguments from a module.

    Args:
        module (module): The module to inspect.
    """
    print(f"Module: {module.__name__}")
    print("Classes:")
    for name, obj in inspect.getmembers(module, inspect.isclass):
        print(f"  {name}")
        print(f"    Documentation: {obj.__doc__}")
        print("    Methods:")
        for method_name, method_obj in inspect.getmembers(obj, inspect.isfunction):
            print(f"      {method_name}")
            print(f"        Documentation: {method_obj.__doc__}")
            print(f"        Arguments: {inspect.signature(method_obj)}")
        print()

    print("Functions:")
    for name, obj in inspect.getmembers(module, inspect.isfunction):
        print(f"  {name}")
        print(f"    Documentation: {obj.__doc__}")
        print(f"    Arguments: {inspect.signature(obj)}")
        print()

# Example usage
import torch

print_module_info(torch)


In [ ]:
import inspect
from typing import Any, Callable, Dict, List, Type

def get_module_info(module: Any, max_depth: int = 1) -> Dict[str, Any]:
    """
    Retrieve information about classes and functions in a module.

    Args:
        module: The module to inspect.
        max_depth: The maximum depth of inspection for nested classes and functions.

    Returns:
        A dictionary containing information about the module's classes and functions.
    """
    module_info = {
        "classes": [],
        "functions": [],
    }

    for name, obj in inspect.getmembers(module):
        if inspect.isclass(obj):
            class_info = {
                "name": obj.__name__,
                "doc": inspect.getdoc(obj),
                "methods": [],
            }
            if max_depth > 0:
                try:
                    for method_name, method in inspect.getmembers(obj, predicate=inspect.isfunction):
                        method_info = {
                            "name": method_name,
                            "doc": inspect.getdoc(method),
                            "args": inspect.signature(method),
                        }
                        class_info["methods"].append(method_info)
                except (ValueError, TypeError):
                    pass
            module_info["classes"].append(class_info)
        elif inspect.isfunction(obj):
            function_info = {
                "name": obj.__name__,
                "doc": inspect.getdoc(obj),
                "args": inspect.signature(obj),
            }
            module_info["functions"].append(function_info)

    return module_info

def print_module_info(module_info: Dict[str, Any]) -> None:
    """
    Print the module information in a readable format.

    Args:
        module_info: A dictionary containing the module information.
    """
    print("Classes:")
    for class_info in module_info["classes"]:
        print(f"  {class_info['name']}:")
        print(f"    Documentation: {class_info['doc']}")
        print("    Methods:")
        for method_info in class_info["methods"]:
            print(f"      {method_info['name']}:")
            print(f"        Documentation: {method_info['doc']}")
            print(f"        Arguments: {method_info['args']}")
        print()

    print("Functions:")
    for function_info in module_info["functions"]:
        print(f"  {function_info['name']}:")
        print(f"    Documentation: {function_info['doc']}")
        print(f"    Arguments: {function_info['args']}")
        print()

# Example usage
import torch
module_info = get_module_info(torch, max_depth=0)
print_module_info(module_info)


In [12]:
import inspect
from typing import Any, Callable, Dict, List, Type

def get_module_info(module: Any, max_depth: int = 1) -> Dict[str, Any]:
    """
    Retrieve information about classes and functions in a module.

    Args:
        module: The module to inspect.
        max_depth: The maximum depth of inspection for nested classes and functions.

    Returns:
        A dictionary containing information about the module's classes and functions.
    """
    module_info = {
        "classes": [],
        "functions": [],
    }

    for name, obj in inspect.getmembers(module):
        if inspect.isclass(obj):
            class_info = {
                "name": obj.__name__,
                "doc": inspect.getdoc(obj),
                "methods": [],
            }
            if max_depth > 0:
                try:
                    for method_name, method in inspect.getmembers(obj, predicate=inspect.isfunction):
                        method_info = {
                            "name": method_name,
                            "doc": inspect.getdoc(method),
                            "args": inspect.signature(method),
                        }
                        class_info["methods"].append(method_info)
                except (ValueError, TypeError):
                    pass
            module_info["classes"].append(class_info)
        elif inspect.isfunction(obj):
            function_info = {
                "name": obj.__name__,
                "doc": inspect.getdoc(obj),
                "args": inspect.signature(obj),
            }
            module_info["functions"].append(function_info)

    return module_info

def write_module_info_to_markdown(module_info: Dict[str, Any], filename: str) -> None:
    """
    Write the module information to a Markdown file.

    Args:
        module_info: A dictionary containing the module information.
        filename: The name of the Markdown file to write to.
    """
    with open(filename, "w") as file:
        file.write("# Module Information\n\n")

        file.write("## Classes\n\n")
        for class_info in module_info["classes"]:
            file.write(f"### {class_info['name']}\n")
            file.write(f"Documentation: {class_info['doc']}\n\n")
            file.write("#### Methods\n\n")
            for method_info in class_info["methods"]:
                file.write(f"##### {method_info['name']}\n")
                file.write(f"Documentation: {method_info['doc']}\n\n")
                file.write(f"Arguments: {method_info['args']}\n\n")
            file.write("\n")

        file.write("## Functions\n\n")
        for function_info in module_info["functions"]:
            file.write(f"### {function_info['name']}\n")
            file.write(f"Documentation: {function_info['doc']}\n\n")
            file.write(f"Arguments: {function_info['args']}\n\n")

# Example usage
import langchain
module_info = get_module_info(langchain, max_depth=200)
write_module_info_to_markdown(module_info, "module_info.md")


In [14]:
import os
from typing import List

def generate_module_structure(modules: List[str], output_file: str) -> None:
    """
    Generate a markdown file with the folder structure and file list for the given modules.

    :param modules: List of module names
    :param output_file: Name of the output markdown file
    """
    with open(output_file, 'w',encoding='utf-8') as file:
        file.write("# Module Structure\n\n")
        for module in modules:
            file.write(f"## {module}\n")
            file.write("```\n")
            file.write(f"{module}/\n")
            file.write(f"├── {module}/\n")
            file.write(f"│   ├── __init__.py\n")
            file.write(f"│   ├── {module}.py\n")
            file.write(f"│   ├── utils.py\n")
            file.write(f"│   └── tests/\n")
            file.write(f"│       ├── __init__.py\n")
            file.write(f"│       └── test_{module}.py\n")
            file.write(f"├── setup.py\n")
            file.write(f"├── README.md\n")
            file.write(f"├── LICENSE\n")
            file.write(f"└── requirements.txt\n")
            file.write("```\n\n")

def main() -> None:
    """
    Main function to get user input and generate the module structure.
    """
    modules = "torch"
    modules = [module.strip() for module in modules]
    output_file = "module_structure.md"
    generate_module_structure(modules, output_file)
    print(f"Module structure generated in {output_file}")

if __name__ == "__main__":
    main()


Module structure generated in module_structure.md


In [16]:
import os
from typing import List

def generate_module_structure(modules: List[str], output_file: str) -> None:
    """
    Generate a markdown file with the folder structure and file list for the given modules.

    This function takes a list of module names and an output file name as parameters.
    It creates a markdown file with the specified name and generates a folder structure
    and file list for each module. The folder structure includes the module directory,
    an __init__.py file, a module Python file, a utils.py file, and a tests directory
    with its own __init__.py and test file. Additionally, it includes setup.py,
    README.md, LICENSE, and requirements.txt files at the root level.

    :param modules: List of module names
    :param output_file: Name of the output markdown file
    """
    with open(output_file, 'w',encoding='utf-8') as file:
        # Write the main heading for the module structure
        file.write("# Module Structure\n\n")

        # Iterate over each module and generate its structure
        for module in modules:
            # Write the module name as a subheading
            file.write(f"## {module}\n")

            # Start a code block to represent the folder structure
            file.write("```\n")

            # Write the module directory
            file.write(f"{module}/\n")

            # Write the files and directories inside the module directory
            file.write(f"├── {module}/\n")
            file.write(f"│   ├── __init__.py\n")
            file.write(f"│   ├── {module}.py\n")
            file.write(f"│   ├── utils.py\n")
            file.write(f"│   └── tests/\n")
            file.write(f"│       ├── __init__.py\n")
            file.write(f"│       └── test_{module}.py\n")

            # Write additional files at the root level
            file.write(f"├── setup.py\n")
            file.write(f"├── README.md\n")
            file.write(f"├── LICENSE\n")
            file.write(f"└── requirements.txt\n")

            # End the code block
            file.write("```\n\n")

def main() -> None:
    """
    Main function to get user input and generate the module structure.

    This function prompts the user to enter module names as a comma-separated string.
    It splits the user input by commas and strips any leading/trailing whitespace from
    each module name. It then calls the generate_module_structure function with the
    list of module names and the output file name. Finally, it prints a message
    indicating that the module structure has been generated in the specified output file.
    """
    # Prompt the user to enter module names
    modules = 'torch'

    # Strip leading/trailing whitespace from each module name
    modules = [module.strip() for module in modules]

    # Specify the output file name
    output_file = "module_structure.md"

    # Call the generate_module_structure function with the module names and output file
    generate_module_structure(modules, output_file)

    # Print a message indicating the successful generation of the module structure
    print(f"Module structure generated in {output_file}")

if __name__ == "__main__":
    main()


Module structure generated in module_structure.md


In [17]:
import os
from typing import List

def generate_module_structure(module_name: str, output_file: str) -> None:
    """
    Generate the structure of a given module in a Markdown file.

    Args:
        module_name (str): The name of the module to generate the structure for.
        output_file (str): The path to the output Markdown file.
    """
    module = __import__(module_name)
    module_path = os.path.dirname(module.__file__)

    with open(output_file, "w") as file:
        file.write(f"# {module_name}\n\n")
        traverse_directory(module_path, file)

def traverse_directory(directory: str, file: os.TextIOWrapper, level: int = 0) -> None:
    """
    Recursively traverse a directory and write its structure to the Markdown file.

    Args:
        directory (str): The directory to traverse.
        file (os.TextIOWrapper): The file object to write the structure to.
        level (int): The indentation level for the current directory.
    """
    indent = "  " * level
    entries = os.listdir(directory)
    folders: List[str] = []
    files: List[str] = []

    for entry in entries:
        if os.path.isdir(os.path.join(directory, entry)):
            folders.append(entry)
        else:
            files.append(entry)

    for folder in folders:
        file.write(f"{indent}- {folder}/\n")
        traverse_directory(os.path.join(directory, folder), file, level + 1)

    for file_name in files:
        file.write(f"{indent}- {file_name}\n")

# Example usage
module_name = "t"
output_file = "torch_structure.md"
generate_module_structure(module_name, output_file)
print(f"Module structure generated in {output_file}")


AttributeError: module 'os' has no attribute 'TextIOWrapper'

In [21]:
import os
from typing import List

def generate_module_structure(module_name: str, output_file: str) -> None:
    """
    Generate the structure of a given module in a Markdown file.

    Args:
        module_name (str): The name of the module to generate the structure for.
        output_file (str): The path to the output Markdown file.
    """
    module = __import__(module_name)
    module_path = os.path.dirname(module.__file__)

    with open(output_file, "w") as file:
        file.write(f"# {module_name}\n\n")
        traverse_directory(module_path, file)

def traverse_directory(directory: str, file, level: int = 0) -> None:
    """
    Recursively traverse a directory and write its structure to the Markdown file.

    Args:
        directory (str): The directory to traverse.
        file (TextIOWrapper): The file object to write the structure to.
        level (int): The indentation level for the current directory.
    """
    indent = "  " * level
    entries = os.listdir(directory)
    folders: List[str] = []
    files: List[str] = []

    for entry in entries:
        if os.path.isdir(os.path.join(directory, entry)):
            folders.append(entry)
        else:
            files.append(entry)

    for folder in folders:
        file.write(f"{indent}- {folder}/\n")
        traverse_directory(os.path.join(directory, folder), file, level + 1)

    for file_name in files:
        file.write(f"{indent}- {file_name}\n")

# Example usage
module_name = "torch"
output_file = "torch_structure.md"
generate_module_structure(module_name, output_file)
print(f"Module structure generated in {output_file}")


Module structure generated in torch_structure.md


In [23]:
import os
from typing import List

def generate_module_structure(module_name: str, output_file: str) -> None:
    """
    Generate the structure of a given module in a Markdown file.

    Args:
        module_name (str): The name of the module to generate the structure for.
        output_file (str): The path to the output Markdown file.
    """
    module = __import__(module_name)
    module_path = os.path.dirname(module.__file__)

    with open(output_file, "w",encoding='utf-8') as file:
        file.write(f"# {module_name} Module Structure\n\n")
        file.write("```\n")
        traverse_directory(module_path, file)
        file.write("```\n")

def traverse_directory(directory: str, file, level: int = 0) -> None:
    """
    Recursively traverse a directory and write its structure to the Markdown file.

    Args:
        directory (str): The directory to traverse.
        file (TextIOWrapper): The file object to write the structure to.
        level (int): The indentation level for the current directory.
    """
    indent = "│   " * level
    entries = os.listdir(directory)
    folders: List[str] = []
    files: List[str] = []

    for entry in entries:
        if os.path.isdir(os.path.join(directory, entry)):
            folders.append(entry)
        else:
            files.append(entry)

    for folder in folders:
        file.write(f"{indent}├── {folder}/\n")
        traverse_directory(os.path.join(directory, folder), file, level + 1)

    for file_name in files:
        file.write(f"{indent}├── {file_name}\n")

    if level > 0:
        file.write(f"{indent[:-4]}└──\n")

# Example usage
module_name = "torch"
output_file = "torch_structure.md"
generate_module_structure(module_name, output_file)
print(f"Module structure generated in {output_file}")


Module structure generated in torch_structure.md


In [ ]:
import os
import inspect
from typing import List

def generate_module_structure(module_name: str, output_file: str) -> None:
    """
    Generate the structure of a given module in a Markdown file.

    Args:
        module_name (str): The name of the module to generate the structure for.
        output_file (str): The path to the output Markdown file.
    """
    module = __import__(module_name)
    module_path = os.path.dirname(module.__file__)

    with open(output_file, "w", encoding='utf-8') as file:
        file.write(f"# {module_name} Module Structure\n\n")
        file.write("```\n")
        traverse_directory(module_path, file, module_name)
        file.write("```\n")

def traverse_directory(directory: str, file, module_name: str, level: int = 0) -> None:
    """
    Recursively traverse a directory and write its structure to the Markdown file.

    Args:
        directory (str): The directory to traverse.
        file (TextIOWrapper): The file object to write the structure to.
        module_name (str): The name of the current module.
        level (int): The indentation level for the current directory.
    """
    indent = "│   " * level
    entries = os.listdir(directory)
    folders: List[str] = []
    files: List[str] = []

    for entry in entries:
        if os.path.isdir(os.path.join(directory, entry)):
            folders.append(entry)
        else:
            files.append(entry)

    for folder in folders:
        file.write(f"{indent}├── {folder}/\n")
        traverse_directory(os.path.join(directory, folder), file, f"{module_name}.{folder}", level + 1)

    for file_name in files:
        if file_name.endswith(".py") and file_name != "__init__.py":
            module_path = os.path.join(directory, file_name)
            module_name_with_file = f"{module_name}.{file_name[:-3]}"
            file.write(f"{indent}├── {file_name}\n")
            write_classes_and_functions(module_name_with_file, file, level + 1)

    if level > 0:
        file.write(f"{indent[:-4]}└──\n")

def write_classes_and_functions(module_name: str, file, level: int) -> None:
    """
    Write the classes and functions of a module to the Markdown file.

    Args:
        module_name (str): The name of the module.
        file (TextIOWrapper): The file object to write the classes and functions to.
        level (int): The indentation level for the classes and functions.
    """
    indent = "│   " * level
    module = __import__(module_name, fromlist=[''])

    for name, obj in inspect.getmembers(module):
        if inspect.isclass(obj):
            file.write(f"{indent}│   Class: {name}\n")
        elif inspect.isfunction(obj):
            file.write(f"{indent}│   Function: {name}\n")

# Example usage
module_name = "torch"
output_file = "torch_structure.md"
generate_module_structure(module_name, output_file)
print(f"Module structure generated in {output_file}")


In [40]:
import os
import inspect
from typing import List

def generate_module_structure(module_name: str, output_file: str) -> None:
    """
    Generate the structure of a given module in a Markdown file.

    Args:
        module_name (str): The name of the module to generate the structure for.
        output_file (str): The path to the output Markdown file.

    Raises:
        ModuleNotFoundError: If the specified module is not found.
        ImportError: If there is an error importing the specified module.
        AttributeError: If the specified module has missing attributes.
    """
    try:
        module = __import__(module_name)
        module_path = os.path.dirname(module.__file__)

        with open(output_file, "w", encoding='utf-8') as file:
            file.write(f"# {module_name} Module Structure\n\n")
            file.write("```\n")
            traverse_directory(module_path, file, module_name)
            file.write("```\n")
    except (ModuleNotFoundError, ImportError, AttributeError):
        print(f"Module '{module_name}' or one of its dependencies not found or has missing attributes. Skipping...")

def traverse_directory(directory: str, file, module_name: str, level: int = 0) -> None:
    """
    Recursively traverse a directory and write its structure to the Markdown file.

    Args:
        directory (str): The directory to traverse.
        file (TextIOWrapper): The file object to write the structure to.
        module_name (str): The name of the current module.
        level (int, optional): The indentation level for the current directory. Defaults to 0.
    """
    indent = "│   " * level
    entries = os.listdir(directory)
    folders: List[str] = []
    files: List[str] = []

    for entry in entries:
        if os.path.isdir(os.path.join(directory, entry)):
            folders.append(entry)
        else:
            files.append(entry)

    for folder in folders:
        file.write(f"{indent}├── {folder}/\n")
        traverse_directory(os.path.join(directory, folder), file, f"{module_name}.{folder}", level + 1)

    for file_name in files:
        if file_name.endswith(".py") and file_name != "__init__.py":
            module_path = os.path.join(directory, file_name)
            module_name_with_file = f"{module_name}.{file_name[:-3]}"
            file.write(f"{indent}├── {file_name}\n")
            write_classes_and_functions(module_name_with_file, file, level + 1)

    if level > 0:
        file.write(f"{indent[:-4]}└──\n")

def write_classes_and_functions(module_name: str, file, level: int) -> None:
    """
    Write the classes and functions of a module to the Markdown file.

    Args:
        module_name (str): The name of the module.
        file (TextIOWrapper): The file object to write the classes and functions to.
        level (int): The indentation level for the classes and functions.

    Raises:
        ModuleNotFoundError: If the specified module is not found.
        ImportError: If there is an error importing the specified module.
        AttributeError: If the specified module has missing attributes.
    """
    indent = "│   " * level
    try:
        module = __import__(module_name, fromlist=[''])

        for name, obj in inspect.getmembers(module):
            if inspect.isclass(obj):
                file.write(f"{indent}│   Class: {name}\n")
            elif inspect.isfunction(obj):
                file.write(f"{indent}│   Function: {name}\n")
    except (ModuleNotFoundError, ImportError, AttributeError):
        file.write(f"{indent}│   (Module or dependency not found or has missing attributes)\n")

# Example usage
module_name = "trl"
output_file = "torch_structure.md"
generate_module_structure(module_name, output_file)
print(f"Module structure generated in {output_file}")


Module structure generated in torch_structure.md


In [43]:
import os
import inspect
from typing import List


def generate_module_structure(module_name: str, output_file: str) -> None:
    """
    Generate the structure of a given module in a Markdown file.

    Args:
        module_name (str): The name of the module to generate the structure for.
        output_file (str): The path to the output Markdown file.

    Raises:
        ModuleNotFoundError: If the specified module is not found.
        ImportError: If there is an error importing the specified module.
        AttributeError: If the specified module has missing attributes.
    """
    try:
        module = __import__(module_name)
        module_path = os.path.dirname(module.__file__)

        with open(output_file, "w", encoding='utf-8') as file:
            file.write(f"# {module_name} Module Structure\n\n")
            file.write("```\n")
            traverse_directory(module_path, file, module_name)
            file.write("```\n")
    except (ModuleNotFoundError, ImportError, AttributeError):
        print(f"Module '{module_name}' or one of its dependencies not found or has missing attributes. Skipping...")


def traverse_directory(directory: str, file, module_name: str, level: int = 0) -> None:
    """
    Recursively traverse a directory and write its structure to the Markdown file.

    Args:
        directory (str): The directory to traverse.
        file (TextIOWrapper): The file object to write the structure to.
        module_name (str): The name of the current module.
        level (int, optional): The indentation level for the current directory. Defaults to 0.
    """
    indent = "│   " * level
    entries = os.listdir(directory)
    folders: List[str] = []
    files: List[str] = []

    for entry in entries:
        if os.path.isdir(os.path.join(directory, entry)):
            folders.append(entry)
        else:
            files.append(entry)

    for folder in folders:
        file.write(f"{indent}├── {folder}/\n")
        traverse_directory(os.path.join(directory, folder), file, f"{module_name}.{folder}", level + 1)

    for file_name in files:
        if file_name.endswith(".py") and file_name != "__init__.py":
            module_path = os.path.join(directory, file_name)
            module_name_with_file = f"{module_name}.{file_name[:-3]}"
            file.write(f"{indent}├── {file_name}\n")
            write_classes_and_functions(module_name_with_file, file, level + 1)

    if level > 0:
        file.write(f"{indent[:-4]}└──\n")


def write_classes_and_functions(module_name: str, file, level: int) -> None:
    """
    Write the classes and functions of a module to the Markdown file.

    Args:
        module_name (str): The name of the module.
        file (TextIOWrapper): The file object to write the classes and functions to.
        level (int): The indentation level for the classes and functions.

    Raises:
        ModuleNotFoundError: If the specified module is not found.
        ImportError: If there is an error importing the specified module.
        AttributeError: If the specified module has missing attributes.
    """
    indent = "│   " * level
    try:
        module = __import__(module_name, fromlist=[''])

        for name, obj in inspect.getmembers(module):
            if inspect.isclass(obj):
                file.write(f"{indent}│   Class: {name}\n")
            elif inspect.isfunction(obj):
                file.write(f"{indent}│   Function: {name}\n")
    except (ModuleNotFoundError, ImportError, AttributeError):
        file.write(f"{indent}│   (Module or dependency not found or has missing attributes)\n")


# Example usage
module_name = "trl"
output_file = "torch_structure.md"
generate_module_structure(module_name, output_file)
print(f"Module structure generated in {output_file}")


Module structure generated in torch_structure.md


In [46]:
import os
import inspect
from typing import List, TextIO


def generate_module_structure(module_name: str, output_file: str) -> None:
    """
    Generate the structure of a specified module in a Markdown file.

    Args:
        module_name (str): The name of the module for which to generate the structure.
        output_file (str): The path to the output Markdown file to write the structure.

    Raises:
        ModuleNotFoundError: If the specified module is not found.
        ImportError: If there is an error importing the module.
        AttributeError: If the module has missing attributes.
    """
    try:
        module = __import__(module_name)
        module_path = os.path.dirname(module.__file__)

        with open(output_file, "w", encoding='utf-8') as file:
            file.write(f"# Structure of the `{module_name}` Module\n\n")
            file.write("```plaintext\n")
            traverse_directory(module_path, file, module_name)
            file.write("```\n")
    except (ModuleNotFoundError, ImportError, AttributeError) as e:
        print(f"Error processing module '{module_name}': {e}. Skipping...")

def traverse_directory(directory: str, file: TextIO, module_name: str, level: int = 0) -> None:
    """
    Recursively traverse a directory and write its structure to a Markdown file.

    Args:
        directory (str): The directory to traverse.
        file (TextIO): The file to write the structure.
        module_name (str): The name of the current module.
        level (int): The indentation level for the current directory (default is 0).
    """
    indent = "│   " * level
    entries = sorted(os.listdir(directory))
    folders, files = [], []

    for entry in entries:
        full_path = os.path.join(directory, entry)
        if os.path.isdir(full_path):
            folders.append(entry)
        elif entry.endswith(".py"):
            files.append(entry)

    for folder in folders:
        file.write(f"{indent}├── {folder}/\n")
        traverse_directory(os.path.join(directory, folder), file, f"{module_name}.{folder}", level + 1)

    for filename in files:
        file.write(f"{indent}├── {filename}\n")
        if filename != "__init__.py":
            write_classes_and_functions(f"{module_name}.{filename[:-3]}", file, level + 1)

def write_classes_and_functions(module_name: str, file: TextIO, level: int) -> None:
    """
    Write the classes and functions of a module to the Markdown file.

    Args:
        module_name (str): The module name.
        file (TextIO): The file to write to.
        level (int): The indentation level for classes and functions.
    """
    indent = "│   " * level
    try:
        module = __import__(module_name, fromlist=[''])
        members = inspect.getmembers(module, lambda member: inspect.isclass(member) or inspect.isfunction(member))

        for name, obj in members:
            type_info = "Class" if inspect.isclass(obj) else "Function"
            file.write(f"```python{indent}│   {type_info}: {name}```\n")
    except (ModuleNotFoundError, ImportError, AttributeError) as e:
        file.write(f"{indent}│   (Error: {e})\n")

# Example usage
module_name = "trl"
output_file = "module_structure12.md"
generate_module_structure(module_name, output_file)
print(f"Module structure generated in {output_file}")


Module structure generated in module_structure12.md


In [50]:
import os
import inspect
from typing import List


def generate_module_structure(module_name: str, output_file: str) -> None:
    """
    Generate the structure of a given module in a Markdown file.

    This function imports the specified module and generates a Markdown file that represents
    the structure of the module. It includes directories, files, classes, and functions.

    Args:
        module_name (str): The name of the module to generate the structure for.
        output_file (str): The path to the output Markdown file.

    Raises:
        ModuleNotFoundError: If the specified module is not found.
        ImportError: If there is an error importing the specified module.
        AttributeError: If the specified module has missing attributes.
    """
    try:
        module = __import__(module_name)
        module_path = os.path.dirname(module.__file__)

        with open(output_file, "w", encoding='utf-8') as file:
            file.write(f"# {module_name} Module Structure\n\n")
            file.write("```\n")
            traverse_directory(module_path, file, module_name)
            file.write("```\n")
    except (ModuleNotFoundError, ImportError, AttributeError):
        print(f"Module '{module_name}' or one of its dependencies not found or has missing attributes. Skipping...")


def traverse_directory(directory: str, file, module_name: str, level: int = 0) -> None:
    """
    Recursively traverse a directory and write its structure to the Markdown file.

    This function traverses the directories and files within the specified directory and
    writes the structure to the Markdown file. It uses indentation to represent the hierarchy
    of directories and files.

    Args:
        directory (str): The directory to traverse.
        file (TextIOWrapper): The file object to write the structure to.
        module_name (str): The name of the current module.
        level (int, optional): The indentation level for the current directory. Defaults to 0.
    """
    indent = "│   " * level
    entries = os.listdir(directory)
    folders: List[str] = []
    files: List[str] = []

    for entry in entries:
        if os.path.isdir(os.path.join(directory, entry)):
            folders.append(entry)
        else:
            files.append(entry)

    for folder in folders:
        file.write(f"{indent}├── {folder}/\n")
        traverse_directory(os.path.join(directory, folder), file, f"{module_name}.{folder}", level + 1)

    for file_name in files:
        if file_name.endswith(".py") and file_name != "__init__.py":
            module_path = os.path.join(directory, file_name)
            module_name_with_file = f"{module_name}.{file_name[:-3]}"
            file.write(f"{indent}├── {file_name}\n")
            write_classes_and_functions(module_name_with_file, file, level + 1)

    if level > 0:
        file.write(f"{indent[:-4]}└──\n")


import inspect
from typing import TextIO

def write_classes_and_functions(module_name: str, file: TextIO, level: int) -> None:
    """
    Write the classes and functions of a module to the Markdown file, including their signatures and docstrings.

    Args:
        module_name (str): The name of the module.
        file (TextIO): The file object to write the classes and functions to.
        level (int): The indentation level for the classes and functions.

    Returns:
        None

    Raises:
        ModuleNotFoundError: If the specified module or one of its dependencies is not found.
        ImportError: If there is an error importing the specified module.
        AttributeError: If the specified module has missing attributes.
    """
    indent = "│   " * level
    try:
        module = __import__(module_name, fromlist=[''])

        for name, obj in inspect.getmembers(module):
            if inspect.isclass(obj) or inspect.isfunction(obj):
                # Retrieve the signature
                try:
                    sig = inspect.signature(obj)
                except ValueError:
                    sig = '(Unable to retrieve signature)'
                
                # Retrieve the docstring
                docstring = inspect.getdoc(obj) or 'No docstring available'
                docstring = ' '.join(docstring.split())  # Clean up whitespace

                # Write the class or function name, signature, and docstring
                file.write(f" ```python{indent}{'Class' if inspect.isclass(obj) else 'Function'}: {name}{sig}```\n")
                file.write(f" ```python{indent}    Doc: {docstring}```\n")
    except (ModuleNotFoundError, ImportError, AttributeError):
        file.write(f"```python{indent}(Module or dependency not found or has missing attributes)```\n")



# Example usage
module_name = "trl"
output_file = "torch_structure.md"
generate_module_structure(module_name, output_file)
print(f"Module structure generated in {output_file}")


Module structure generated in torch_structure.md


In [51]:
import os
import inspect
from typing import List, TextIO


def generate_module_structure(module_name: str, output_file: str) -> None:
    """
    Generate the structure of a given module in a Markdown file.

    This function imports the specified module and generates a Markdown file that represents
    the structure of the module. It includes directories, files, classes, and functions.

    Args:
        module_name (str): The name of the module to generate the structure for.
        output_file (str): The path to the output Markdown file.

    Raises:
        ModuleNotFoundError: If the specified module is not found.
        ImportError: If there is an error importing the specified module.
        AttributeError: If the specified module has missing attributes.
    """
    try:
        module = __import__(module_name)
        module_path = os.path.dirname(module.__file__)

        with open(output_file, "w", encoding='utf-8') as file:
            file.write(f"# {module_name} Module Structure\n\n")
            file.write("```\n")
            traverse_directory(module_path, file, module_name)
            file.write("```\n")
    except (ModuleNotFoundError, ImportError, AttributeError) as e:
        print(f"Error: {str(e)}. Skipping...")


def traverse_directory(directory: str, file: TextIO, module_name: str, level: int = 0) -> None:
    """
    Recursively traverse a directory and write its structure to the Markdown file.

    This function traverses the directories and files within the specified directory and
    writes the structure to the Markdown file. It uses indentation to represent the hierarchy
    of directories and files.

    Args:
        directory (str): The directory to traverse.
        file (TextIOWrapper): The file object to write the structure to.
        module_name (str): The name of the current module.
        level (int, optional): The indentation level for the current directory. Defaults to 0.
    """
    indent = "│   " * level
    entries = os.listdir(directory)
    folders: List[str] = []
    files: List[str] = []

    for entry in entries:
        if os.path.isdir(os.path.join(directory, entry)):
            folders.append(entry)
        else:
            files.append(entry)

    for folder in folders:
        file.write(f"{indent}├── {folder}/\n")
        traverse_directory(os.path.join(directory, folder), file, f"{module_name}.{folder}", level + 1)

    for file_name in files:
        if file_name.endswith(".py") and file_name != "__init__.py":
            module_path = os.path.join(directory, file_name)
            module_name_with_file = f"{module_name}.{file_name[:-3]}"
            file.write(f"{indent}├── {file_name}\n")
            write_classes_and_functions(module_name_with_file, file, level + 1)

    if level > 0:
        file.write(f"{indent[:-4]}└──\n")


def write_classes_and_functions(module_name: str, file: TextIO, level: int) -> None:
    """
    Write the classes and functions of a module to the Markdown file, including their signatures and docstrings.

    Args:
        module_name (str): The name of the module.
        file (TextIO): The file object to write the classes and functions to.
        level (int): The indentation level for the classes and functions.

    Raises:
        ModuleNotFoundError: If the specified module or one of its dependencies is not found.
        ImportError: If there is an error importing the specified module.
        AttributeError: If the specified module has missing attributes.
    """
    indent = "│   " * level
    try:
        module = __import__(module_name, fromlist=[''])

        for name, obj in inspect.getmembers(module):
            if inspect.isclass(obj) or inspect.isfunction(obj):
                # Retrieve the signature
                try:
                    sig = inspect.signature(obj)
                except ValueError:
                    sig = '(Unable to retrieve signature)'
                
                # Retrieve the docstring
                docstring = inspect.getdoc(obj) or 'No docstring available'
                docstring = ' '.join(docstring.split())  # Clean up whitespace

                # Write the class or function name, signature, and docstring
                file.write(f"{indent}{'Class' if inspect.isclass(obj) else 'Function'}: {name}{sig}\n")
                file.write(f"{indent}    Doc: {docstring}\n")
    except (ModuleNotFoundError, ImportError, AttributeError) as e:
        file.write(f"{indent}(Error: {str(e)})\n")


# Example usage
module_name = "trl"
output_file = "torch_structure.md"
generate_module_structure(module_name, output_file)
print(f"Module structure generated in {output_file}")


Module structure generated in torch_structure.md


In [52]:
import os
import inspect
from typing import List, TextIO


def generate_module_structure(module_name: str, output_file: str) -> None:
    """
    Generate the structure of a given module in a Markdown file.

    This function imports the specified module and generates a Markdown file that represents
    the structure of the module. It includes directories, files, classes, and functions.

    Args:
        module_name (str): The name of the module to generate the structure for.
        output_file (str): The path to the output Markdown file.

    Raises:
        ModuleNotFoundError: If the specified module is not found.
        ImportError: If there is an error importing the specified module.
        AttributeError: If the specified module has missing attributes.
    """
    try:
        module = __import__(module_name)
        module_path = os.path.dirname(module.__file__)

        with open(output_file, "w", encoding='utf-8') as file:
            file.write(f"# {module_name} Module Structure\n\n")
            file.write("```\n")
            traverse_directory(module_path, file, module_name)
            file.write("```\n")
    except (ModuleNotFoundError, ImportError, AttributeError) as e:
        print(f"Error: {str(e)}. Skipping...")


def traverse_directory(directory: str, file: TextIO, module_name: str, level: int = 0) -> None:
    """
    Recursively traverse a directory and write its structure to the Markdown file.

    This function traverses the directories and files within the specified directory and
    writes the structure to the Markdown file. It uses indentation to represent the hierarchy
    of directories and files.

    Args:
        directory (str): The directory to traverse.
        file (TextIOWrapper): The file object to write the structure to.
        module_name (str): The name of the current module.
        level (int, optional): The indentation level for the current directory. Defaults to 0.
    """
    indent = "│   " * level
    entries = os.listdir(directory)
    folders: List[str] = []
    files: List[str] = []

    for entry in entries:
        if os.path.isdir(os.path.join(directory, entry)):
            folders.append(entry)
        else:
            files.append(entry)

    for folder in folders:
        file.write(f"{indent}├── {folder}/\n")
        traverse_directory(os.path.join(directory, folder), file, f"{module_name}.{folder}", level + 1)

    for file_name in files:
        if file_name.endswith(".py") and file_name != "__init__.py":
            module_path = os.path.join(directory, file_name)
            module_name_with_file = f"{module_name}.{file_name[:-3]}"
            file.write(f"{indent}├── {file_name}\n")
            write_classes_and_functions(module_name_with_file, file, level + 1)

    if level > 0:
        file.write(f"{indent[:-4]}└──\n")


def write_classes_and_functions(module_name: str, file: TextIO, level: int) -> None:
    """
    Write the classes and functions of a module to the Markdown file, including their signatures and docstrings.

    Args:
        module_name (str): The name of the module.
        file (TextIO): The file object to write the classes and functions to.
        level (int): The indentation level for the classes and functions.

    Raises:
        ModuleNotFoundError: If the specified module or one of its dependencies is not found.
        ImportError: If there is an error importing the specified module.
        AttributeError: If the specified module has missing attributes.
    """
    indent = "│   " * level
    try:
        module = __import__(module_name, fromlist=[''])

        for name, obj in inspect.getmembers(module):
            if inspect.isclass(obj) or inspect.isfunction(obj):
                # Retrieve the signature
                try:
                    sig = inspect.signature(obj)
                except ValueError:
                    sig = '(Unable to retrieve signature)'
                
                # Retrieve the docstring
                docstring = inspect.getdoc(obj) or 'No docstring available'
                docstring = ' '.join(docstring.split())  # Clean up whitespace

                # Write the class or function name, signature, and docstring
                file.write(f"{indent}{'Class' if inspect.isclass(obj) else 'Function'}: {name}{sig}\n")
                file.write(f"{indent}    Doc: {docstring}\n")
    except (ModuleNotFoundError, ImportError, AttributeError) as e:
        file.write(f"{indent}(Error: {str(e)})\n")


# Example usage
module_name = "trl"
output_file = "torch_structure.md"
generate_module_structure(module_name, output_file)
print(f"Module structure generated in {output_file}")


Module structure generated in torch_structure.md


In [53]:
import os
import inspect
from typing import List, TextIO


def generate_module_structure(module_name: str, output_file: str) -> None:
    """
    Generate the structure of a given module in a Markdown file.

    This function imports the specified module and generates a Markdown file that represents
    the structure of the module. It includes directories, files, classes, and functions.

    Args:
        module_name (str): The name of the module to generate the structure for.
        output_file (str): The path to the output Markdown file.

    Raises:
        ModuleNotFoundError: If the specified module is not found.
        ImportError: If there is an error importing the specified module.
        AttributeError: If the specified module has missing attributes.
    """
    try:
        module = __import__(module_name)
        module_path = os.path.dirname(module.__file__)

        with open(output_file, "w", encoding='utf-8') as file:
            file.write(f"# {module_name} Module Structure\n\n")
            traverse_directory(module_path, file, module_name)
    except (ModuleNotFoundError, ImportError, AttributeError) as e:
        print(f"Error: {str(e)}. Skipping...")


def traverse_directory(directory: str, file: TextIO, module_name: str, level: int = 0) -> None:
    """
    Recursively traverse a directory and write its structure to the Markdown file.

    This function traverses the directories and files within the specified directory and
    writes the structure to the Markdown file. It uses indentation to represent the hierarchy
    of directories and files.

    Args:
        directory (str): The directory to traverse.
        file (TextIOWrapper): The file object to write the structure to.
        module_name (str): The name of the current module.
        level (int, optional): The indentation level for the current directory. Defaults to 0.
    """
    indent = "  " * level
    entries = os.listdir(directory)
    folders: List[str] = []
    files: List[str] = []

    for entry in entries:
        if os.path.isdir(os.path.join(directory, entry)):
            folders.append(entry)
        else:
            files.append(entry)

    for folder in folders:
        file.write(f"{indent}- {folder}/\n")
        traverse_directory(os.path.join(directory, folder), file, f"{module_name}.{folder}", level + 1)

    for file_name in files:
        if file_name.endswith(".py") and file_name != "__init__.py":
            module_path = os.path.join(directory, file_name)
            module_name_with_file = f"{module_name}.{file_name[:-3]}"
            file.write(f"{indent}- {file_name}\n")
            write_classes_and_functions(module_name_with_file, file, level + 1)


def write_classes_and_functions(module_name: str, file: TextIO, level: int) -> None:
    """
    Write the classes and functions of a module to the Markdown file, including their signatures and docstrings.

    Args:
        module_name (str): The name of the module.
        file (TextIO): The file object to write the classes and functions to.
        level (int): The indentation level for the classes and functions.

    Raises:
        ModuleNotFoundError: If the specified module or one of its dependencies is not found.
        ImportError: If there is an error importing the specified module.
        AttributeError: If the specified module has missing attributes.
    """
    indent = "  " * level
    try:
        module = __import__(module_name, fromlist=[''])

        for name, obj in inspect.getmembers(module):
            if inspect.isclass(obj) or inspect.isfunction(obj):
                # Retrieve the signature
                try:
                    sig = inspect.signature(obj)
                except ValueError:
                    sig = '(Unable to retrieve signature)'
                
                # Retrieve the docstring
                docstring = inspect.getdoc(obj) or 'No docstring available'
                docstring = ' '.join(docstring.split())  # Clean up whitespace

                # Write the class or function name, signature, and docstring
                file.write(f"{indent}- {'Class' if inspect.isclass(obj) else 'Function'}: `{name}{sig}`\n")
                file.write(f"{indent}  - Description: {docstring}\n")
    except (ModuleNotFoundError, ImportError, AttributeError) as e:
        file.write(f"{indent}- (Error: {str(e)})\n")


# Example usage
module_name = "trl"
output_file = "torch_structure.md"
generate_module_structure(module_name, output_file)
print(f"Module structure generated in {output_file}")


Module structure generated in torch_structure.md


In [54]:
import os
import inspect
from typing import List, TextIO


def generate_module_structure(module_name: str, output_file: str) -> None:
    """
    Generate the structure of a given module in a Markdown file.

    This function imports the specified module and generates a Markdown file that represents
    the structure of the module. It includes directories, files, classes, and functions.

    Args:
        module_name (str): The name of the module to generate the structure for.
        output_file (str): The path to the output Markdown file.

    Raises:
        ModuleNotFoundError: If the specified module is not found.
        ImportError: If there is an error importing the specified module.
        AttributeError: If the specified module has missing attributes.
    """
    try:
        module = __import__(module_name)
        module_path = os.path.dirname(module.__file__)

        with open(output_file, "w", encoding='utf-8') as file:
            file.write(f"# {module_name} Module Structure\n\n")
            traverse_directory(module_path, file, module_name)
    except (ModuleNotFoundError, ImportError, AttributeError) as e:
        print(f"Error generating module structure: {str(e)}")


def traverse_directory(directory: str, file: TextIO, module_name: str, level: int = 0) -> None:
    """
    Recursively traverse a directory and write its structure to the Markdown file.

    This function traverses the directories and files within the specified directory and
    writes the structure to the Markdown file. It uses indentation to represent the hierarchy
    of directories and files.

    Args:
        directory (str): The directory to traverse.
        file (TextIOWrapper): The file object to write the structure to.
        module_name (str): The name of the current module.
        level (int, optional): The indentation level for the current directory. Defaults to 0.
    """
    indent = "  " * level
    entries = os.listdir(directory)
    folders: List[str] = []
    files: List[str] = []

    for entry in entries:
        if os.path.isdir(os.path.join(directory, entry)):
            folders.append(entry)
        else:
            files.append(entry)

    for folder in folders:
        file.write(f"{indent}- {folder}/\n")
        traverse_directory(os.path.join(directory, folder), file, f"{module_name}.{folder}", level + 1)

    for file_name in files:
        if file_name.endswith(".py") and file_name != "__init__.py":
            module_path = os.path.join(directory, file_name)
            module_name_with_file = f"{module_name}.{file_name[:-3]}"
            file.write(f"{indent}- {file_name}\n")
            write_classes_and_functions(module_name_with_file, file, level + 1)


def write_classes_and_functions(module_name: str, file: TextIO, level: int) -> None:
    """
    Write the classes and functions of a module to the Markdown file.

    This function writes the classes and functions of the specified module to the Markdown file,
    including their signatures and docstrings. It handles cases where the module or its dependencies
    are not found or have missing attributes.

    Args:
        module_name (str): The name of the module.
        file (TextIO): The file object to write the classes and functions to.
        level (int): The indentation level for the classes and functions.
    """
    indent = "  " * level
    try:
        module = __import__(module_name, fromlist=[''])

        for name, obj in inspect.getmembers(module):
            if inspect.isclass(obj) or inspect.isfunction(obj):
                # Retrieve the signature
                try:
                    sig = inspect.signature(obj)
                except ValueError:
                    sig = '(Unable to retrieve signature)'
                
                # Retrieve the docstring
                docstring = inspect.getdoc(obj) or 'No docstring available'
                docstring = ' '.join(docstring.split())  # Clean up whitespace

                # Write the class or function name, signature, and docstring
                file.write(f"{indent}- {'Class' if inspect.isclass(obj) else 'Function'}: {name}{sig}\n")
                file.write(f"{indent}  - Doc: {docstring}\n")
    except (ModuleNotFoundError, ImportError, AttributeError):
        file.write(f"{indent}(Module or dependency not found or has missing attributes)\n")


# Example usage
module_name = "trl"
output_file = "torch_structure.md"
generate_module_structure(module_name, output_file)
print(f"Module structure generated in {output_file}")


Module structure generated in torch_structure.md


In [55]:
import os
import inspect
from typing import List, TextIO


def generate_module_structure(module_name: str, output_file: str) -> None:
    """
    Generate the structure of a given module in a Markdown file.

    This function imports the specified module and generates a Markdown file that represents
    the structure of the module. It includes directories, files, classes, and functions.

    Args:
        module_name (str): The name of the module to generate the structure for.
        output_file (str): The path to the output Markdown file.

    Raises:
        ModuleNotFoundError: If the specified module is not found.
        ImportError: If there is an error importing the specified module.
        AttributeError: If the specified module has missing attributes.
    """
    try:
        module = __import__(module_name)
        module_path = os.path.dirname(module.__file__)

        with open(output_file, "w", encoding='utf-8') as file:
            file.write(f"# {module_name} Module Structure\n\n")
            traverse_directory(module_path, file, module_name)
    except (ModuleNotFoundError, ImportError, AttributeError) as e:
        print(f"Error: {str(e)}. Skipping...")


def traverse_directory(directory: str, file: TextIO, module_name: str, level: int = 0) -> None:
    """
    Recursively traverse a directory and write its structure to the Markdown file.

    This function traverses the directories and files within the specified directory and
    writes the structure to the Markdown file. It uses indentation to represent the hierarchy
    of directories and files.

    Args:
        directory (str): The directory to traverse.
        file (TextIOWrapper): The file object to write the structure to.
        module_name (str): The name of the current module.
        level (int, optional): The indentation level for the current directory. Defaults to 0.
    """
    indent = "  " * level
    entries = os.listdir(directory)
    folders: List[str] = []
    files: List[str] = []

    for entry in entries:
        if os.path.isdir(os.path.join(directory, entry)):
            folders.append(entry)
        else:
            files.append(entry)

    for folder in folders:
        file.write(f"{indent}- {folder}/\n")
        traverse_directory(os.path.join(directory, folder), file, f"{module_name}.{folder}", level + 1)

    for file_name in files:
        if file_name.endswith(".py") and file_name != "__init__.py":
            module_path = os.path.join(directory, file_name)
            module_name_with_file = f"{module_name}.{file_name[:-3]}"
            file.write(f"{indent}- {file_name}\n")
            write_classes_and_functions(module_name_with_file, file, level + 1)


def write_classes_and_functions(module_name: str, file: TextIO, level: int) -> None:
    """
    Write the classes and functions of a module to the Markdown file, including their signatures and docstrings.

    Args:
        module_name (str): The name of the module.
        file (TextIO): The file object to write the classes and functions to.
        level (int): The indentation level for the classes and functions.

    Raises:
        ModuleNotFoundError: If the specified module or one of its dependencies is not found.
        ImportError: If there is an error importing the specified module.
        AttributeError: If the specified module has missing attributes.
    """
    indent = "  " * level
    try:
        module = __import__(module_name, fromlist=[''])

        for name, obj in inspect.getmembers(module):
            if inspect.isclass(obj) or inspect.isfunction(obj):
                # Retrieve the signature
                try:
                    sig = inspect.signature(obj)
                except ValueError:
                    sig = '(Unable to retrieve signature)'
                
                # Retrieve the docstring
                docstring = inspect.getdoc(obj) or 'No docstring available'
                docstring = ' '.join(docstring.split())  # Clean up whitespace

                # Write the class or function name, signature, and docstring
                file.write(f"{indent}- {'Class' if inspect.isclass(obj) else 'Function'}: `{name}{sig}`\n")
                file.write(f"{indent}  - Description: {docstring}\n")
    except (ModuleNotFoundError, ImportError, AttributeError) as e:
        file.write(f"{indent}- (Error: {str(e)})\n")


# Example usage
module_name = "trl"
output_file = "torch_structure.md"
generate_module_structure(module_name, output_file)
print(f"Module structure generated in {output_file}")


Module structure generated in torch_structure.md


In [61]:
import os
import inspect
from typing import List, TextIO


def generate_module_structure(module_name: str, output_file: str, exclude_dirs: List[str] = None, exclude_files: List[str] = None) -> None:
    """
    Generate the structure of a given module in a Markdown file.

    This function imports the specified module and generates a Markdown file that represents
    the structure of the module. It includes directories, files, classes, and functions.

    Args:
        module_name (str): The name of the module to generate the structure for.
        output_file (str): The path to the output Markdown file.
        exclude_dirs (List[str], optional): A list of directory names to exclude from the structure. Defaults to None.
        exclude_files (List[str], optional): A list of file names to exclude from the structure. Defaults to None.

    Raises:
        ModuleNotFoundError: If the specified module is not found.
        ImportError: If there is an error importing the specified module.
        AttributeError: If the specified module has missing attributes.
    """
    try:
        module = __import__(module_name)
        module_path = os.path.dirname(module.__file__)

        with open(output_file, "w", encoding='utf-8') as file:
            file.write(f"# {module_name} Module Structure\n\n")
            traverse_directory(module_path, file, module_name, exclude_dirs, exclude_files)
    except (ModuleNotFoundError, ImportError, AttributeError) as e:
        print(f"Error: {str(e)}. Skipping...")


def traverse_directory(directory: str, file: TextIO, module_name: str, exclude_dirs: List[str] = None, exclude_files: List[str] = None, level: int = 0) -> None:
    """
    Recursively traverse a directory and write its structure to the Markdown file.

    This function traverses the directories and files within the specified directory and
    writes the structure to the Markdown file. It uses indentation to represent the hierarchy
    of directories and files.

    Args:
        directory (str): The directory to traverse.
        file (TextIOWrapper): The file object to write the structure to.
        module_name (str): The name of the current module.
        exclude_dirs (List[str], optional): A list of directory names to exclude from the structure. Defaults to None.
        exclude_files (List[str], optional): A list of file names to exclude from the structure. Defaults to None.
        level (int, optional): The indentation level for the current directory. Defaults to 0.
    """
    indent = "  " * level
    entries = os.listdir(directory)
    folders: List[str] = []
    files: List[str] = []

    for entry in entries:
        if os.path.isdir(os.path.join(directory, entry)):
            if exclude_dirs is None or entry not in exclude_dirs:
                folders.append(entry)
        else:
            if exclude_files is None or entry not in exclude_files:
                files.append(entry)

    for folder in folders:
        file.write(f"{indent}- {folder}/\n")
        traverse_directory(os.path.join(directory, folder), file, f"{module_name}.{folder}", exclude_dirs, exclude_files, level + 1)

    for file_name in files:
        if file_name.endswith(".py") and file_name != "__init__.py":
            module_path = os.path.join(directory, file_name)
            module_name_with_file = f"{module_name}.{file_name[:-3]}"
            file.write(f"{indent}- {file_name}\n")
            write_classes_and_functions(module_name_with_file, file, level + 1)


def write_classes_and_functions(module_name: str, file: TextIO, level: int) -> None:
    """
    Write the classes and functions of a module to the Markdown file, including their signatures, docstrings, and source code.

    Args:
        module_name (str): The name of the module.
        file (TextIO): The file object to write the classes and functions to.
        level (int): The indentation level for the classes and functions.

    Raises:
        ModuleNotFoundError: If the specified module or one of its dependencies is not found.
        ImportError: If there is an error importing the specified module.
        AttributeError: If the specified module has missing attributes.
    """
    indent = "  " * level
    try:
        module = __import__(module_name, fromlist=[])

        for name, obj in inspect.getmembers(module):
            if inspect.isclass(obj) or inspect.isfunction(obj):
                # Skip built-in classes
                if inspect.isbuiltin(obj):
                    continue
                
                # Retrieve the signature
                try:
                    sig = inspect.signature(obj)
                except ValueError:
                    sig = '(Unable to retrieve signature)'
                
                # Retrieve the docstring
                docstring = inspect.getdoc(obj) or 'No docstring available'
                docstring = ' '.join(docstring.split())  # Clean up whitespace

                # Retrieve the source code
                try:
                    source_code = inspect.getsource(obj)
                except OSError:
                    source_code = 'Source code not available'

                # Write the class or function name, signature, docstring, and source code
                file.write(f"{indent}  -- {'Class' if inspect.isclass(obj) else 'Function'}: `{name}{sig}`\n")
                file.write(f"{indent}  -- Description: {docstring}\n")
                file.write(f"{indent}  -- Source Code:\n")
                file.write(f"```python\n{source_code}\n```\n")
    except (ModuleNotFoundError, ImportError, AttributeError) as e:
        file.write(f"{indent}- (Error: {str(e)})\n")

# Example usage
module_name = "langchain"
output_file = "torch_structure.md"
exclude_dirs = ["_internal", "csrc"]  # Exclude specific directories
exclude_files = ["setup.py"]  # Exclude specific files
generate_module_structure(module_name, output_file, exclude_dirs, exclude_files)
print(f"Module structure generated in {output_file}")


Module structure generated in torch_structure.md
